In [1]:
#import all the necessory libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from mpl_toolkits import mplot3d

### Logistic regression  

In [2]:
root="/home/subhash/Desktop/spam/nnfl assignment/"
data=pd.read_excel(os.path.join(root,'data3.xlsx'),header=None)
data=data.to_numpy()
np.random.shuffle(data)
data[:,-1]=data[:,-1]-1
print(data.shape)
#now we obtain the shuffled set of data convinient to split

(100, 5)


In [3]:
def split_classes(data):
    y=data[:,-1]
    class0=[]
    class1=[]
    classes=[]
    for insts in range(0,data.shape[0]):
        if (y[insts]==0):
            class0.append(data[insts])
        elif (y[insts]==1):
            class1.append(data[insts])
    c0=np.array(class0)
    c1=np.array(class1)
    classes.append(c0)
    classes.append(c1)
    return classes

In [4]:
def split_train_test(classes,train_perc):
    train=[]
    test=[]
    class_insts=int(30)
    for cls in range(0,len(classes)):
        np.random.shuffle(np.array(classes[cls]))
        traintemp=classes[cls][0:class_insts,:]
        train.append(traintemp)
        testtemp=classes[cls][class_insts:,:]
        test.append(testtemp)
    train=np.array(train).reshape(((class_insts*2),5))
    np.random.shuffle(train)
    test=np.array(test).reshape((40,5))
    np.random.shuffle(test)
    return train,test


In [5]:
#splitting the data into features and class value
def split_x_y(data):
    y=(data[:,-1]).reshape((data.shape[0],1))
    x=np.delete(data,-1,1)
    return x,y

In [6]:
classes=split_classes(data)
train,test=split_train_test(classes,60)
x_train,y_train=split_x_y(train)
x_test,y_test=split_x_y(test)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60, 4)
(60, 1)
(40, 4)
(40, 1)


In [7]:
#sigmoid function, takes in a single numerical value
def sigmoid(x):
    y=1/(1+np.exp(-x))
    return y
#returns a single numerical value

In [8]:
def hypothesis(w,x):
    hyps=[]
    for insts in range(0,x.shape[0]):
        h=np.dot(w.T,x[insts])
        hyp=sigmoid(h)
        hyps.append(hyp)
    return hyps

In [9]:
#likelyhood function
def likely_func(hyps,y):
    likes=[]
    for insts in range(0,y.shape[0]):
        like=(y[insts]*np.log(hyps[insts]))+((1-y[insts])*np.log(1-hyps[insts]))
        likes.append(like)
    return(np.array(likes))

In [10]:
def costavg(errors):
    costav=np.mean(errors)
    return(costav)

In [11]:
#derivative of likelyhood function
def dervlikef(hyps,y,x):
    deriv=[]
    for insts in range(0,x.shape[0]):
        hyps[insts]=np.reshape((hyps[insts]),(1,1))
        derv=np.dot(np.reshape((x[insts]),(1,4)).T,((y[insts]*(1-hyps[insts])-((1-y[insts])*(hyps[insts])))))
        deriv.append(derv)
        dW=np.squeeze(np.mean(deriv,axis=0,keepdims=True)/2)
        dW=np.reshape(dW,(x.shape[1],1))
    return dW

In [12]:
def update_parameters(w,dervlikef,alpha):
    w=w+(alpha*dervlikef)
    return w

In [17]:
def train(x_train,y_train,alpha,iters,x_test,y_test):
    w=np.ones([4,1])
    w_list=[]
    for k in range(iters):
        for insts in range(0,x_train.shape[0]):
            w_list.append(w)
            deriv=dervlikef(hypothesis(w,x_train),y_train,x_train)
            w=update_parameters(w,deriv,alpha)
        y_cap=hypothesis(w,x_test)
        for insts in range(0,np.array(y_cap).shape[0]):
            if (y_cap[insts]<0.5):
                y_cap[insts]=0
            else:
                y_cap[insts]=1
        y_cap=np.array(y_cap).reshape(y_test.shape)
        true_pos=0
        true_neg=0
        false_pos=0
        false_neg=0
        for insts in range(0,y_test.shape[0]):
            if (y_test[insts]==1):
                if (y_cap[insts]==1):
                    true_pos=true_pos+1
                else:
                    false_neg=false_neg+1
            else:
                if (y_cap[insts]==0):
                    true_neg=true_neg+1
                else:
                    false_pos=false_pos+1
        sensitivity=true_pos/(true_pos+false_neg)
        specificity=true_neg/(true_neg+false_pos)
        accuracy=(true_pos+true_neg)/(true_pos+true_neg+false_pos+false_neg)
    return sensitivity,specificity,accuracy

In [18]:
se,sp,acc=train(x_train,y_train,0.05,1,x_test,y_test)


In [19]:
print(se)
print(sp)
print(acc)

1.0
1.0
1.0
